In [6]:
%load_ext autoreload
%autoreload 2
from main import Config, load_data
from seqmodels import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
## Space to explore
# two types of encoders
# batch size
# lr
# size of input seq and output seq
# 

In [ ]:
## Output Summarization for error analytics
## given a sequence -> reduce to output
    # contains add to cart 
    # contains checkout 
    # loops between pages 
    # contains successful checkout 
    # faces issues after doing ... 
    # 

## Input processing and Embedding

In [7]:
# set config and read graph data in gretel format (will be re-written later)
DOMAIN = 'pollini'
config, DEVICE = load_config(DOMAIN)
graph, trajectories, pairwise_node_features, _ = load_data(config)

## Model Building

In [ ]:
hidden_size = 512 #HP
dp = 0.1 #HP
lr = 0.0025 #HP
tf = 0.95 #HP

n_node = graph.n_node
encoder = EncoderRNN(n_node+2, hidden_size).to(DEVICE)
attn_decoder = AttnDecoderRNN(hidden_size, n_node+2, dropout_p=dp).to(DEVICE)
models = trainIters(trajectories[0], encoder, attn_decoder, trajectories[2],
                    n_node+1, n_node, n_epoch=5, learning_rate=lr, tf=tf)

100%|██████████| 19887/19887 [04:19<00:00, 76.64it/s]


Epoch: 0 	 Loss: 1.1930257188021967


100%|██████████| 4990/4990 [00:27<00:00, 180.75it/s]
/home/vikramreddyardham/code/gretel-path-extrapolation/seqmodels.py:413: RuntimeWarning: invalid value encountered in true_divide
  node_accuracy = node_accuracy/node_counts
  0%|          | 7/19887 [00:00<04:50, 68.50it/s]

0.024363198156083718 [0.11486486 0.10805687        nan 0.27076591        nan        nan
 0.34606841        nan 0.34606205        nan 0.48489543 0.125
 0.04464286 0.27925697 0.                nan 0.         0.
        nan 0.35378715 0.         0.23157895 0.26106195        nan
        nan 0.                nan 0.2806574  0.         0.
 0.21522922 0.03100775 0.06779661        nan 0.                nan
        nan] [0.45691383 0.18617234 0.11362725 0.08176353 0.05971944 0.03446894
 0.01923848 0.00460922 0.         0.         0.         0.        ]


100%|██████████| 19887/19887 [04:28<00:00, 73.93it/s]


...

  0%|          | 7/19887 [00:00<04:51, 68.17it/s]

0.029981121631366577 [0.16666667 0.24259006        nan 0.32307692        nan        nan
 0.35278631        nan 0.42773893        nan 0.3642794  0.
 0.         0.40634006 0.                nan 0.08988764 0.04639175
        nan 0.24479167 0.         0.25457571 0.23759542        nan
        nan 0.04191617        nan 0.27448609 0.         0.10526316
 0.26498423 0.06374502 0.09452736        nan 0.                nan
        nan] [0.50798722 0.23642173 0.12819489 0.09704473 0.06948882 0.03214856
 0.01038339 0.00399361 0.         0.         0.         0.        ]


100%|██████████| 19887/19887 [04:22<00:00, 75.63it/s]


Epoch: 4 	 Loss: 0.2725837894096221


 10%|▉         | 490/4984 [00:02<00:22, 199.39it/s]

## Save models

In [ ]:
model_params = dict({"hidden_size": hidden_size,"n_node":n_node, "n_in" : n_in, "n_out":n_out, "dp":dp, "lr":lr})
model_file = "./models/"+DOMAIN+"_model.pt"
torch.save({"params": model_params,**models}, model_file)

In [ ]:
temp = dict({"encoder" : models["encoder"].state_dict(),
             "decoder" : models["decoder"].state_dict(), 
             "optim_encoder" : models["optim_encoder"].state_dict(), 
             "optim_decoder": models["optim_decoder"].state_dict()})
torch.save({"params":model_params, **temp}, model_file)

## Metrics and Evaluation

In [ ]:
training_pairs = tensors_from_paths(trajectories[2], n_in, n_out)
print(evaluateRandomly(encoder, attn_decoder, training_pairs,"fancy_grid", n_out=n_out))

In [ ]:
import plotly.express as px
lengths = []

for i in trajectories[2]:
    lengths.append(len(i))
px.histogram(lengths, cumulative=True, histnorm='probability')

## Sequence metrics

+ k step-accuracy (accuracy of the kth step)
+ k-step overlap (overlap of predictions and actual)
+ Node accuracy (how many times a node is predicted right)
   - Presence
   - Position
+ bleu-score

In [ ]:
class PathPredict(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, n_lstm, bid=False):
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_lstm, bidirectional=bid)
        self.h0 = torch.rand(self.num_layers, hidden_dim)  # batch_size?
        self.c0 = torch.rand(self.num_layers, hidden_dim)  # batch_size?
        self.linear = nn.Linear(n_emb*seq_len, n_out)
        self.activation = nn.Sigmoid()

    def foward(self, X):
        emb = self.embedding(X) #[seq_len, bs]
        x = self.lstm(emb) #[seq_len, bs, n_emb]
        y = self.activation(self.linear(x.view(-1, 1)))


In [ ]:
n_hid = 4
n_inp = 3
seq_len = 10
num_layers = 2
bid = 1

lstm = nn.LSTM(n_inp, n_hid, num_layers)
x = torch.rand(seq_len, 1, n_inp)
h0 = torch.rand(num_layers, 1, n_hid)
c0 = torch.rand(num_layers, 1, n_hid)
y = lstm(x, (h0,c0))

In [ ]:
## lengths distribution 